In [ ]:
import os

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

tf.config.set_visible_devices([], 'GPU') # it's needed to be done on CPU due to low VRAM

# Constants for image size and other parameters
img_width, img_height = 192, 192
img_folder = "img"
model = tf.keras.models.load_model("resnet50_2layers.h5")

# Function to preprocess the image
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img_array = cv2.resize(img, (192, 192))
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to get activation maps for a given layer and image
def get_activation_maps(model, layer_index, list_of_img_array):
    layer_outputs = [layer.output for layer in model.layers]
    activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(np.array(list_of_img_array), verbose=0)
    return activations[layer_index]

# Function to plot activation maps
def plot_activation_maps(all_activation_maps, layer_name, path_to_save):
    print(all_activation_maps.shape)
    maps, _, _, n_filters = all_activation_maps.shape

    plt.figure(figsize=(552, 552))
    for m in range(maps):
        for i in range(n_filters):
            plt.subplot(92, 92, m*n_filters + 1 + i)
            plt.imshow(all_activation_maps[m, :, :, i], cmap="viridis")
            plt.axis("off")
    plt.suptitle(f"Activation Maps for Layer: {layer_name}", fontsize=14)
    plt.tight_layout()
    plt.savefig(path_to_save+layer_name+".png")

def plot_activation_maps_dense(all_activation_maps, layer_name, path_to_save):
    print(all_activation_maps.shape)
    plt.figure(figsize=(20, 2))
    plt.imshow(all_activation_maps, cmap="hot")
    plt.axis("off")
    plt.suptitle(f"Activation Maps for Layer: {layer_name}", fontsize=14)
    plt.tight_layout()
    plt.savefig(path_to_save+layer_name+".png")


# Function to get all image file paths from a folder
def get_image_paths(folder_path):
    image_paths = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_paths.append(os.path.join(folder_path, filename))
    return image_paths


def plot_arrays(arrays, colors, number):
    plt.figure(figsize=(12, max(1, max(arr.shape[1] for arr in arrays)//3)))
    plt.rcParams["axes.facecolor"] = "white"

    for i, array in enumerate(arrays):
        number_of_filters = array.shape[1] if len(array.shape) > 1 else 1
        for filter_num in range(number_of_filters):
            x = array[:, filter_num] if len(array.shape) > 1 else array
            y = np.full_like(x, filter_num + 1)
            plt.scatter(x, y, color=colors[i], s=50, alpha=0.6)

    plt.xlabel("Value (0 to 255)")
    plt.ylabel("Number of Filter")
    plt.title("Plot of Arrays")
    plt.grid(color="none")
    plt.yticks(range(1, max(arr.shape[1] for arr in arrays) + 1))
    plt.ylim(0.5, max(arr.shape[1] for arr in arrays) + 0.5)
    plt.savefig(str(number)+"_fig.png")

def get_image_paths_from_folder(folder_path):
    image_paths = []
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(root, filename)
                image_paths.append(img_path)
    return image_paths

def plot_arrays_with_stats(arrays, colors, layer, numb_top_filtr=6):
    number_of_filters = arrays[0].shape[1]

    if number_of_filters == 1:
        plt.figure(figsize=(12, 1))
        plt.rcParams["axes.facecolor"] = "white"
        for j, array in enumerate(arrays):
            x = array
            y = np.full_like(x, 1)
            marker_size = 100
            plt.scatter(x, y, color=colors[j], s=marker_size, alpha=0.6)
        for j, array in enumerate(arrays):
            mean_val = np.mean(array)
            std_val = np.std(array)
            plt.axvline(mean_val, color=colors[j], linestyle="dashed",
                        linewidth=3, label=f"Mean (Array {j+1}): {mean_val:.2f}")
            plt.axvline(mean_val + std_val, color=colors[j], linestyle='dotted',
                        linewidth=2, label=f"Std Deviation (Array {j+1}): {std_val:.2f}")
            plt.axvline(mean_val - std_val,
                        color=colors[j], linestyle="dotted", linewidth=2)
        plt.yticks([])
        plt.grid(color="none")
        plt.xlabel("Activation maps value")
        plt.savefig("banana_"+layer+'activation_map.png')
        return 0

    fig, axs = plt.subplots(number_of_filters, 1, figsize=(
        12, 2 * number_of_filters), sharex=True)
    plt.rcParams["axes.facecolor"] = "white"

    for i in range(number_of_filters):
        for j, array in enumerate(arrays):
            x = array[:, i]
            y = np.full_like(x, 1)
            marker_size = 100
            axs[i].scatter(x, y, color=colors[j], s=marker_size, alpha=0.6)
        for j, array in enumerate(arrays):
            mean_val = np.mean(array[:, i])
            std_val = np.std(array[:, i])
            axs[i].axvline(mean_val, color=colors[j], linestyle="dashed",
                           linewidth=3, label=f"Mean (Array {j+1}): {mean_val:.2f}")
            axs[i].axvline(mean_val + std_val, color=colors[j], linestyle="dotted",
                           linewidth=2, label=f"Std Deviation (Array {j+1}): {std_val:.2f}")
            axs[i].axvline(mean_val - std_val, color=colors[j],
                           linestyle="dotted", linewidth=2)
        axs[i].set_ylabel(f"F{i+1}")
        axs[i].set_yticklabels([])
        axs[i].grid(color="none")
        difference = np.mean(arrays[0][:, i]) - np.mean(arrays[1][:, i])

    axs[-1].set_xlabel("Activation maps value")
    plt.savefig("banana_"+layer+'activation_map.png')


folder_true = r"C:\Users\PC\OneDrive\Pulpit\baza danych\images\real"
folder_AI = r"C:\Users\PC\OneDrive\Pulpit\baza danych\images\ai"
folder_true_banana = r"C:\Users\PC\OneDrive\Pulpit\baza danych\images\banana_real"
folder_false_banana = r"C:\Users\PC\OneDrive\Pulpit\baza danych\images\banana_fake"

true_paths = get_image_paths_from_folder(folder_true)
AI_paths = get_image_paths_from_folder(folder_AI)
true_banana_paths = get_image_paths_from_folder(folder_true_banana)
fake_banana_paths = get_image_paths_from_folder(folder_false_banana)

images_array_true = [preprocess_image(img) for img in true_paths]
images_array_AI = [preprocess_image(img) for img in AI_paths]
images_array_banana_true = [preprocess_image(img) for img in true_banana_paths]
images_array_banana_fake = [preprocess_image(img) for img in fake_banana_paths]

all_layers = [layer for layer in model.layers]
for i, layer in enumerate(all_layers):
    if "dense" in layer.name:
        print("\n", layer.name)
        activation_maps_true = get_activation_maps(model, i, np.squeeze(images_array_true, axis=1))
        print("activation_maps_true", activation_maps_true.shape)
        activation_maps_AI = get_activation_maps(model, i, np.squeeze(images_array_AI, axis=1))
        print("activation_maps_AI", activation_maps_AI.shape)
        activation_maps_banana_true = get_activation_maps(model, i, np.squeeze(images_array_banana_true, axis=1))
        print("activation_maps_banana_true", activation_maps_banana_true.shape)
        activation_maps_banana_fake = get_activation_maps(model, i, np.squeeze(images_array_banana_fake, axis=1))
        print("activation_maps_banana_fake", activation_maps_banana_fake.shape)

        plot_arrays_with_stats([activation_maps_true, activation_maps_AI, activation_maps_banana_true, activation_maps_banana_fake], ['green', 'red', 'orange', 'blue'], layer.name)